In [82]:
import geopandas as gpd
import requests
import json

In [3]:
def get_flight_limit(area):
    if "red" in area.lower():
        return 0
    elif "orange" in area.lower():
        return 25
    elif "yellow" in area.lower():
        return 45
    elif "blue" in area.lower():
        return 60
    elif "green" in area.lower():
        return 120
    else:
        return -1

In [5]:
try:
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
except:
    pass


In [9]:
restriction_areas = gpd.read_file('flight_restriction_areas.kml', driver='KML')
restriction_areas = restriction_areas.drop(columns=['Description'])
# restriction_areas["boundary"] = restriction_areas.boundary

limits = []
for area in restriction_areas["Name"]:
    limits.append(get_flight_limit(area))

restriction_areas["flight_limit"] = limits
restriction_areas

,Name,geometry,flight_limit
0,Yellow area,"POLYGON Z ((8.92387 44.40400 0.00000, 8.94172 ...",45
1,Red area,"POLYGON Z ((8.97801 44.40574 0.00000, 8.96061 ...",0
2,Yellow area 2,"POLYGON Z ((8.95307 44.41890 0.00000, 8.94270 ...",45


In [11]:
# Generate random points within each polygon
points = restriction_areas.sample_points(100)
points.explore(m=restriction_areas.explore(), color='red')

In [55]:
# Parse the points coordinates to make a string for the request to open elevation API
def parse_points(points):
    coords = []
    msg = []

    for p in points:
        coords.append(list(p.coords))

    for i in range(len(coords)):
        coords[i] = [(x[1], x[0]) for x in coords[i]]
    
    for c in coords:
        msg.append(str(c[0][0]) + ',' + str(c[0][1]))
    
    msg = '|'.join(msg)

    return msg

In [56]:
payloads = []
for p in points:
    payloads.append(parse_points(list(p.geoms)))
payloads


['44.40508229867618,8.924863562506639|44.40381681075411,8.925153633450725|44.406169699178754,8.925825953284042|44.40816952429158,8.926335438176352|44.40412268081243,8.926752307944167|44.40482719512408,8.92719025974249|44.4043483586103,8.927257644019601|44.40554598796891,8.928205757674238|44.40580268063698,8.928311733591338|44.40855112205741,8.92893102548416|44.4040756620894,8.929038751136737|44.40629477056507,8.929230670690323|44.40381222314004,8.929256203893754|44.40393565173565,8.929481619346033|44.41008140078986,8.92951539108645|44.40646888849924,8.929578673754813|44.40487385251461,8.929790903565868|44.41511304811274,8.929831071223434|44.41262293626072,8.9298616833156|44.403290675523316,8.930077373426409|44.40994672338835,8.930445710629538|44.41392912118207,8.931515056263128|44.415088724648186,8.931728357108357|44.40256828731041,8.931847123522214|44.40448369444693,8.931934720572102|44.41112775080246,8.931975831317951|44.404907357964895,8.932246465592891|44.40436944224353,8.932449370

In [74]:
def get_elevation(coords):
    api_url = 'https://api.open-elevation.com/api/v1/lookup?locations='
    url = api_url + coords
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            # Get all elevations and return
            # elevations = []
            # for result in data['results']:
            #     elevations.append(result['elevation'])
            return data
        else:
            print("No results found for the provided coordinates.")
    else:
        print("Error occurred while fetching data:", response.status_code)

In [85]:

# Call the function with the coordinates string
# Create a json file

data = {}

for i in range(len(payloads)):
    data[restriction_areas['Name'][i]] = get_elevation(payloads[i])['results']

with open("elevations.json", "w") as outfile:
    outfile.write(json.dumps(data, indent=4))
